# import libraries

In [29]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
import ast
from scipy.spatial.distance import cosine, euclidean, hamming
from sklearn.preprocessing import normalize
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from time import time
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

# read data

## Read Music Data

In [30]:
data = pd.read_csv('/home/mohamed/spotify recommender system/data/data.csv')
genre_data = pd.read_csv('/home/mohamed/spotify recommender system/data/data_by_genres.csv')
year_data = pd.read_csv('/home/mohamed/spotify recommender system/data/data_by_year.csv')
data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [31]:
data['artists'] = data['artists'].map(lambda x: x.lstrip('[').rstrip(']'))
data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"'Sergei Rachmaninoff', 'James Levine', 'Berlin...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,'Dennis Day',0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,'KHP Kridhamardawa Karaton Ngayogyakarta Hadin...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,'Frank Parker',0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,'Phil Regan',0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [32]:
data['artists'] = data['artists'].map(lambda x: x[1:-1])
data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"Sergei Rachmaninoff', 'James Levine', 'Berline...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,Dennis Day,0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,Frank Parker,0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,Phil Regan,0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


## Read Music Playlist Data
### The original dataset is quite large. I only read 1% of rows for faster run.

In [33]:
p = 0.02  # to randomly select 2% of the rows
df_playlist = pd.read_csv('/home/mohamed/spotify recommender system/spotify playlists/spotify_dataset.csv', on_bad_lines='skip', skiprows=lambda i: i>0 and random.random() > p)
df_playlist.head()


,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Crowded House,Not The Girl You Think You Are,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Cocktail Slippers,St. Valentine's Day Massacre,HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tom Petty And The Heartbreakers,American Girl,IOW 2012
3,07f0fc3be95dcd878966b1f9572ff670,2080,The Backup,2080
4,07f0fc3be95dcd878966b1f9572ff670,Bonobo,Black Sands - Duke Dumont's 'Grains Of Sand' R...,Chill out


### Clean up column names


In [34]:
df_playlist.columns = df_playlist.columns.str.replace('"', '')
df_playlist.columns = df_playlist.columns.str.replace('name', '')
df_playlist.columns = df_playlist.columns.str.replace(' ', '')
df_playlist.columns

Index(['user_id', 'artist', 'track', 'playlist'], dtype='object')

## For recommender system, I'm only keeping the artists with frequency higher than 50

In [35]:
df_playlist = df_playlist.groupby('artist').filter(lambda x : len(x)>=50)

### And keeping the users with at least 10 unique artists in their playlists to lessen the impact of cold start problem

In [36]:
df_playlist = df_playlist[df_playlist.groupby('user_id').artist.transform('nunique') >= 10]

### group by to get the frequnecy count for each user and artist (# of times that an artist has appeared in playlists created by a user)

In [37]:
size = lambda x: len(x)
df_freq = df_playlist.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

,user_id,artist,freq
1571,db937456654d2465292c4daa947c95de,Johnny Cash,29
830,7ee2b92c5bcf6133b8132363e5bda960,Frank Zappa,23
13,00123e0f544dee3ab006aa7f1e5725a7,Tom Waits,13
9,00123e0f544dee3ab006aa7f1e5725a7,Neil Young,13
1222,a251c8204b5bc479dc5c83c15b7cc929,The Beach Boys,13


### Create a DF for artists and add artist id

In [38]:
df_artist = pd.DataFrame(df_freq["artist"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
df_artist.head()

,artist_id,artist
0,0,Johnny Cash
1,1,Frank Zappa
2,2,Tom Waits
3,3,Neil Young
4,4,The Beach Boys


### Get the Information of user

In [39]:
def GetInPut(user):
    inputArtist = pd.DataFrame(user)
    #Filtering out the movies by title
    Id = df_artist[df_artist['artist'].isin(inputArtist['artist'].tolist())]
    #Then merging it so we can get the movieId. It's implicitly merging it by title.
    inputArtist = pd.merge(Id, inputArtist)
    #Dropping information we won't use from the input dataframe
    #inputArtist = inputArtist.drop('year', 1)
    return inputArtist

In [40]:
user = [
            {'artist':'Ella Fitzgerald', 'freq':40},
            {'artist':'Frank Sinatra', 'freq':10},
            {'artist':'Lil Wayne', 'freq':3},
            {'artist':"The Rolling Stones", 'freq':5},
            {'artist':'Louis Armstrong', 'freq':5}
         ] 

In [41]:
inputArtist = GetInPut(user)

## Collaborative Filtering Song Recommendation 




**Similarity of users to input user**
Next, we are going to compare all users  to our specified user and find the one that is most similar.
we're going to find out how similar each user is to the input through the Pearson Correlation Coefficient. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation") 

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [42]:
def ColFilter(inputArtist,df_freq):
    #Filtering out the movies by title
    Id = df_artist[df_artist['artist'].isin(inputArtist['artist'].tolist())]
    #Then merging it so we can get the movieId. It's implicitly merging it by title.
    inputArtist = pd.merge(Id, inputArtist)
    #Dropping information we won't use from the input dataframe
    #inputArtist = inputArtist.drop('year', 1)
    df_freq  = pd.merge(df_freq , df_artist, how='inner', on='artist')
    userSubsetGroup = df_freq.groupby(['user_id'])
    userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
    userSubsetGroup = userSubsetGroup[0:100]
    #Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
    pearsonCorDict = {}
    #For every user group in our subset
    for name, group in userSubsetGroup:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='artist_id')
        inputArtist = inputArtist.sort_values(by='artist_id')
        #Get the N for the formula
        n = len(group)
        #Get the review scores for the movies that they both have in common
        temp = inputArtist[inputArtist['artist_id'].isin(group['artist_id'].tolist())]
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp['freq'].tolist()
        #put the current user group reviews in a list format
        tempGroupList = group['freq'].tolist()
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)
        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearsonCorDict[name] = 0
    pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['user_id'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
    topUsersRating=topUsers.merge(df_freq, left_on='user_id', right_on='user_id', how='inner')
    topUsersRatingS=topUsersRating
    topUsersRating['weightedFreq'] = topUsersRating['similarityIndex']*topUsersRating['freq']
    #Applies a sum to the topUsers after grouping it up by userId
    tempTopUsersRating = topUsersRating.groupby('artist_id').sum()[['similarityIndex','weightedFreq']]
    tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedFreq']
    #Creates an empty dataframe
    recommendation_df = pd.DataFrame()
    #Now we take the weighted average
    recommendation_df['weighted average freq score'] = tempTopUsersRating['sum_weightedFreq']/tempTopUsersRating['sum_similarityIndex']
    recommendation_df['artist_id'] = tempTopUsersRating.index
    # Sort the recomendation by the weighted average freq score
    recommendation_df = recommendation_df.sort_values(by='weighted average freq score', ascending=False)
    recommendation_final = df_artist.loc[df_artist['artist_id'].isin(recommendation_df.head(10)['artist_id'].tolist())]
    return recommendation_final, topUsersRatingS


In [43]:
recommendation_final, topUsersRatingS = ColFilter(inputArtist, df_freq)
recommendation_final

,artist_id,artist


In [44]:
topUsersRatingS

,similarityIndex,user_id,artist,freq,artist_id,weightedFreq


## Get the final recommendation of artists

In [45]:
df_playlist_2  = pd.merge(df_playlist , df_artist, how='inner', on='artist')
df_playlist_2.head()

,user_id,artist,track,playlist,artist_id
0,c50566d83fba17b20697039d5824db78,Fleetwood Mac,Caroline,Annat,61
1,55f361fad00af15e96e790bc360ce1c6,Fleetwood Mac,Dreams,vintage summer,61
2,e9cd5d794fa122de70c7714f61a38c0a,Fleetwood Mac,"Brushes (Never Going Back Again) [Sessions, Ro...",best of radio paradise,61
3,84d3144396e1404debb8a63448edd8ca,Fleetwood Mac,Don't Stop - Remastered LP Version,feel good songs,61
4,42de1f6c71a4ee9074abdc094fa8a163,Fleetwood Mac,Say You Love Me,Fleetwood Mac – Fleetwood Mac,61


# Prepare the music data for Content-based Filtering

In [46]:
data['song_id']=data.index
df = data[['danceability','energy',"valence","speechiness","instrumentalness","acousticness"]]
df.index = data['song_id']
# normalized  data by columns
df_normalized = pd.DataFrame(normalize(df, axis=1))
df_normalized.columns = df.columns
df_normalized.index = df.index
df_normalized.head()

,danceability,energy,valence,speechiness,instrumentalness,acousticness
song_id,,,,,,
0,0.204439,0.154611,0.043526,0.026819,0.643359,0.719566
1,0.526206,0.219092,0.618725,0.266637,0.000000,0.470308
2,0.238274,0.120590,0.028622,0.024627,0.663245,0.698114
3,0.258165,0.290084,0.154899,0.033233,0.000026,0.907802
4,0.382654,0.176680,0.231606,0.034787,0.000002,0.876076


## Content-based Filtering Song Recommendation

In [47]:
"""
Music Recommender based on different distance calculation approaches

df_normalized: normalized song data
distance_method: distance calculation approach: e.g. cosine, euclidean, hamming
song_id: find similar songs based on the selected song
N: Top N song(s)

return 1) song data of selected spng and Top N recommendation, 
2) song id and song name of Top N recommendation
"""

def Content_filter_music_recommender(song_id, N):
    distance_method = cosine
    allSongs = pd.DataFrame(df_normalized.index)
    allSongs = allSongs[allSongs.song_id != song_id]
    allSongs["distance"] = allSongs["song_id"].apply(lambda x: distance_method(df_normalized.loc[song_id], df_normalized.loc[x]))
    # sort by distance then recipe id, the smaller value of recipe id will be picked. 
    TopNRecommendation = allSongs.sort_values(["distance"]).head(N).sort_values(by=['distance', 'song_id'])
    #print(data['name'].loc[song_id, :])
    Recommendation = pd.merge(TopNRecommendation , data, how='inner', on='song_id')
    SongName = Recommendation['name']
    return SongName

## get recommendations for a song with ID 3, and requesting the top 5 recommendations.
## The resulting variable SongName should contain the names of these recommended songs.

In [48]:
SongName=Content_filter_music_recommender(3, 5)
SongName

0                     Pause Track - Live
1    StaggerLee Has His Day at the Beach
2                            Pause Track
3                           Silent Track
4                           Magic Window
Name: name, dtype: object